In [25]:
import bs4
import base64
import re
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
import pandas as pd
import os
from tqdm import tqdm_notebook

In [26]:
def tokenize(s):
    #print(s)aq
    s = ' '.join(re.split('https?://.*\\W', s))
    return list(filter(lambda x: len(x) > 1, re.split('\\W', s)))

In [33]:
def links_from_doc(doc):
    links = []
    for link in doc.find_all('a'):
        if link.has_attr('href'):
            href = link.get('href')
            if href.startswith('http'):
                links.append(href)
    return links

def text_from_doc(doc):
    words = []
    if isinstance(doc, bs4.element.Comment):
        pass
    elif isinstance(doc, bs4.element.Tag):
        if (doc.name != 'style' and doc.name != 'script'):
            for child in doc.contents:
                words += text_from_doc(child)
    else:
        words += tokenize(doc)
    return words

def parse_doc(doc):
    return text_from_doc(doc), links_from_doc(doc)

In [28]:
def get_lemmas(words):
    result = !echo {words} | /Users/xrater/Documents/mystem -lnd
    return result

In [29]:
files_directory = "../byweb_for_course/"
files_in_dir = !(ls {files_directory} | grep byweb.0)

documents = dict()
documents_meta = dict()
for file_name in files_in_dir:
    data_file = open(os.path.join('../byweb_for_course/', file_name), 'r') 
    lines = data_file.readlines()
    
#    for i in tqdm_notebook(range(1, len(lines) - 1, 3)):
    for i in range(1, 5, 3):
        page = lines[i] + lines[i + 1] + lines[i + 2]
        document = bs4.BeautifulSoup(page, "lxml")
        docid = document.docid.contents[0]
        url = base64.b64decode(document.docurl.contents[0]).decode("cp1251")
        content = base64.b64decode(document.content.contents[0]).decode("cp1251")
        page_data = bs4.BeautifulSoup(content, 'lxml')

        documents[url] = page_data
        documents_meta[url] = dict()
        
    data_file.close()

In [34]:
all_links = dict()
frequencies = dict()

for url in tqdm_notebook(documents):
    document = documents[url]
    meta = documents_meta[url]

    words, links = parse_doc(document)
    all_links[url] = links

    meta["words_count"] = len(words)
    meta["text_size"] = sum(list(map(len, words)))
    lemmas = get_lemmas(words)
        
    for lemma in lemmas:
        if lemma not in frequencies:
            frequencies[lemma] = 0
        frequencies[lemma] += 1

In [8]:
results = dict()
words = []
words_number = 0
summary_words_len = 0.0
for word in frequencies:
    words.append(word)
    words_number += frequencies[word]
    summary_words_len += frequencies[word] * len(word)
results["dictionary_size"] = len(words)
results["average_dictionary_word_len"] = np.average(list(map(len, words)))
results["average_collection_word_len"] = summary_words_len / words_number

results

{'dictionary_size': 283,
 'average_dictionary_word_len': 6.865724381625442,
 'average_collection_word_len': 6.283898305084746}

In [47]:
import networkx as nx
from urllib.parse import urlparse

def is_outer_link(source, dest):
    return source.netloc != dest.netloc
    
G = nx.DiGraph()
for source_url in all_links:
    for dest_url in all_links[source_url]:
        source = urlparse(source_url)
        dest = urlparse(dest_url)
        if source.netloc != dest.netloc:
            G.add_edge(source, dest)
nx.write_gexf(G, './graph.gexf')

In [123]:
kek = dict()
for ln in wordlens:
    key = (ln // 100) * 100
    if key not in kek:
        kek[key] = 1
    else:
        kek[key] = kek[key] + 1

In [34]:
df = pd.DataFrame()
df["keys"] = kek.keys()
df["values"] = kek.values()

px.bar(df, x="keys", y="values")

AttributeError: 'tuple' object has no attribute 'keys'

In [106]:
a = ' '.join(docs[7][2])
b = !echo $a | /home/mikhail/mystem

In [91]:
docs[7][1]

'http://brest.linux.by/forum/index.php?action=vthread&forum=8&topic=126'

In [56]:
s = 'abacaba http://krk.lol/arbidol caa'
re.split('https?://.*\\W', s)

['abacaba ', 'caa']

In [92]:
re.split('({|})', b[0])

['Начало']

In [93]:
b[0]

'Начало'

In [94]:
re.split('(^|}).*?{', b[0])

['Начало']

In [107]:
b

['Начало{начало|начинать}Ответить{отвечать}Статистика{статистика|статистик}Регистрация{регистрация}Поиск{поиск}FAQ{FAQ??}Язык{язык}Аноним{аноним}thxburjuinrwz{thxburjuinrwz??}html{html??}html{html??}html{html??}html{html??}html{html??}html{html??}html{html??}Аноним{аноним}mziwj{mziwj??}html{html??}html{html??}html{html??}html{html??}html{html??}html{html??}html{html??}html{html??}Аноним{аноним}mziwj{mziwj??}']

In [117]:
kek = re.split('{|}', b[0])
kek1 = ''
for i in range(1, len(kek), 2):
    kek1 += re.split('\|', kek[i])[0] + ' '

In [ ]:
kek1

In [13]:
text="<div>Hello</div>"
document = bs4.BeautifulSoup(text)
len(document)

1

In [35]:
all_links

{'http://forum.linux.by/viewtopic.php?t=6497': ['http://www.linux.by/index.php',
  'http://forum.linux.by/viewforum.php?f=24',
  'http://forum.linux.by/staff.php',
  'http://www.linux.by/index.php',
  'http://www.linux.by/wiki/',
  'http://forum.linux.by/album.php',
  'http://forum.linux.by/',
  'http://www.linux.by/index.php?op=1',
  'http://forum.linux.by/viewforum.php?f=26',
  'http://www.linux.by/index.php?op=2',
  'http://forum.linux.by/links.php',
  'http://forum.linux.by/search.php',
  'http://forum.linux.by/docs/rules.html',
  'http://forum.linux.by/logs.php',
  'http://forum.4minsk.com',
  'http://www.google.com/',
  'http://beloffice.by',
  'http://wwp.icq.com/scripts/search.dll?to=152391401',
  'http://www.gnome.org/projects/NetworkManager/',
  'http://wwp.icq.com/scripts/search.dll?to=300237142',
  'http://www.avilink.net',
  'http://www.phpbb.com/',
  'http://br.by',
  'http://top.list.ru/jump?from=266764',
  'http://catalog.aport.ru/rus/'],
 'http://forum.linux.by/viewtop